In [14]:
from keras import backend as K

def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score

In [15]:
#1) 기본 딥러닝으로 학습성과 예측 모델 구현하기
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt
import os


# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.set_random_seed(seed)

# 데이터 입력
df_pre = pd.read_csv('./dataset/0706_2.csv', header=None)
df = df_pre.sample(frac=0.3)
print(df_pre.info())

#28831건 컬럼 total 12 columns 0~11


#2) data 설정
dataset = df.values
X = dataset[:,0:11]
Y = dataset[:,11]

# 훈련 데이터와 테스트 데이터로 분리하기
X_train, X_test, y_train, y_test = train_test_split(X, Y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28832 entries, 0 to 28831
Data columns (total 12 columns):
0     28832 non-null int64
1     28832 non-null int64
2     28832 non-null int64
3     28832 non-null float64
4     28832 non-null int64
5     28832 non-null int64
6     28832 non-null int64
7     28832 non-null int64
8     28832 non-null float64
9     28832 non-null float64
10    28832 non-null float64
11    28832 non-null int64
dtypes: float64(4), int64(8)
memory usage: 2.6 MB
None


In [17]:
# 모델 설정
model = Sequential() 
model.add(Dense(30, input_dim=11, activation='relu'))
model.add(Dense(9, activation='relu'))
model.add(Dense(1, activation='softmax'))

#모델 컴파일
model.compile(loss='binary_crossentropy',
           optimizer='adam',
          metrics=['accuracy', precision, recall, f1score])


# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"


# 모델 업데이트 및 저장
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)


#에포크가 진행되면서 모든 값이 저장되는 것이 아니라 테스트 오차를 실행한 결괏값이 향상되었을 때만 저장됨.
df = df_pre.sample(frac=1)

# 모델 실행 및 저장
history =model.fit(X_train, y_train, validation_split=0.2, epochs=200, batch_size=200, verbose=0, callbacks=[checkpointer])






Epoch 00001: val_loss improved from inf to 3.66012, saving model to ./model/01-3.6601.hdf5

Epoch 00002: val_loss did not improve

Epoch 00003: val_loss did not improve

Epoch 00004: val_loss did not improve

Epoch 00005: val_loss did not improve

Epoch 00006: val_loss did not improve

Epoch 00007: val_loss did not improve

Epoch 00008: val_loss did not improve

Epoch 00009: val_loss did not improve

Epoch 00010: val_loss did not improve

Epoch 00011: val_loss did not improve

Epoch 00012: val_loss did not improve

Epoch 00013: val_loss did not improve

Epoch 00014: val_loss did not improve

Epoch 00015: val_loss did not improve

Epoch 00016: val_loss did not improve

Epoch 00017: val_loss did not improve

Epoch 00018: val_loss did not improve

Epoch 00019: val_loss did not improve

Epoch 00020: val_loss did not improve

Epoch 00021: val_loss did not improve

Epoch 00022: val_loss did not improve

Epoch 00023: val_loss did not improve

Epoch 00024: val_loss did not improve

Epoch 0002

In [20]:

# 결과 출력
print("\n Accuracy: %.4f" % (model.evaluate(X_train, y_train)[1]))

_loss, _acc, _precision, _recall, _f1score = model.evaluate(X_test, y_test, batch_size=200, verbose=0)
print('loss: {:.3f}, accuracy: {:.3f}, precision: {:.3f}, recall: {:.3f}, f1score: {:.3f}'.format(_loss, _acc, _precision, _recall, _f1score))





6487/6487 [==============================] - 0s 15us/step

 Accuracy: 0.7558
loss: 3.707, accuracy: 0.767, precision: 0.767, recall: 1.000, f1score: 0.868
